<img src="Archivos/miad4.png" width=800x>

## Interactividad usando `panel`

En este taller podrás poner en práctica los conceptos aprendidos acerca del uso del paquete `panel`. Usarás `panel` con el fin de crear dashboards como herramienta de apoyo al momento de resolver problemas específicos en un contexto de Analytics.

Instrucciones y condiciones generales: Esta es una actividad calificada y, por lo tanto, debe ser resuelta individualmente.

### Al desarrollar este taller podrás verificar tu progreso en:

1. Programar la funcionalidad de un dashboard en `panel`.
2. Agregar gráficos y widgets para hacer visualizaciones agradables en `panel`.
3. Personalizar dashobards para generar distintas visualizaciones interactivas en `panel`.

In [1]:
import panel as pn
import plotly.graph_objects as go
import pandas as pd
pn.extension('plotly')

## Pregunta 1

Con la base de datos *COVID-19.csv* crea un dashboard en `panel` que:

1. Permita crear un diagrama de barras horizontal que muestre los casos de COVID-19 de acuerdo a la selección de los siguentes criterios:<br />
    a. Casos o muertes.<br />
    b. Periodo: Histórico, Últimos 7 días y Últimas 24 horas.<br />
    Pista: utilice un control de tipo `RadioButtonGroup` para cada uno de los dos criterios.<br />
2. Contenga un diagrama de barras horizontal de las muertes por millón de personas.
3. Permita visualizar a tráves de un diagrama de barras horizontal los casos en cada una de las regiones de la Organización Munidal de la Salud.

Nota: a excpeción de las regiones, muestre los 10 mayores valores en cada gráfico.

| <center>Variable</center> |<center>Descripción</center> | 
| :-- | :-- | 
| Name | País |
| WHO Region | Región de la OMS |
| Cases - cumulative total | Total de casos hasta el momento |
| Cases - cumulative total per 1 million population | Número de casos acumulado por millón de personas |
| Cases - newly reported in last 7 days | Número de casos nuevos en los últimos 7 días |
| Cases - newly reported in last 24 hours | Número de casos nuevos en las últimas 24 horas |
| Deaths - cumulative total | Total de muertes hasta el momento |
| Deaths - cumulative total per 1 million population | Número de muertes acumulado por millón de personas |
| Deaths - newly reported in last 7 days | Número de muertes nuevas en los últimos 7 días |
| Deaths - newly reported in last 24 hours | Número de muertes nuevas en las últimas 24 horas |

In [2]:
# YOUR CODE HERE
datos = pd.read_csv('Archivos/COVID-19.csv') 
datos.head()

,Name,WHO Region,Cases - cumulative total,Cases - cumulative total per 1 million population,Cases - newly reported in last 7 days,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 1 million population,Deaths - newly reported in last 7 days,Deaths - newly reported in last 24 hours,Transmission Classification
0,United States of America,Americas,26547977,80204.73,871365,129961,455735,1376.83,22562,3549,Community transmission
1,India,South-East Asia,10826363,7845.17,80180,12059,154996,112.32,722,78,Clusters of cases
2,Brazil,Americas,9447165,44444.82,328652,50872,230034,1082.21,7368,1239,Community transmission
3,Russian Federation,Europe,3967281,27185.36,116842,16048,76661,525.31,3479,432,Clusters of cases
4,The United Kingdom,Europe,3929839,57888.79,133747,18262,112092,1651.18,6521,828,Community transmission


In [3]:
# YOUR CODE HERE
titulo = '# Tablero de seguimiento COVID-19'

In [4]:
estadistica = pn.widgets.RadioButtonGroup(
    name='Estadistica', options=['Casos', 'Muertes'], button_type='success')

periodo = pn.widgets.RadioButtonGroup(
    name='Periodo', options=['Histórico', 'Últimos 7 días','Últimas 24 horas'], button_type='success')

In [5]:
# YOUR CODE HERE
pd.options.mode.chained_assignment = None  # default='warn'
df = datos

@pn.depends(estadistica.param.value,periodo.param.value)

def funcion_interactiva(estadistica,periodo): 
    col = list(df.columns)
   
    #Seleccion del dataframe para la primera grafica
    if estadistica == 'Casos':
        cases_filt = [col for col in df if col.find('Cases')>=0]
        cases_filt.insert(0, 'Name')
        df1 = df[cases_filt] 
    else:
        deaths_filt = [col for col in df if col.find('Deaths')>=0]
        deaths_filt.insert(0, 'Name')
        df1 = df[deaths_filt]
        
    col_drop = [col for col in df1 if col.find('per 1 million population')>=0]
    df1 = df1.drop(col_drop, axis =1)
    
    
    if periodo == 'Histórico':
        period_filt = [col for col in df1 if col.find('cumulative')>=0]
        period_filt.insert(0, 'Name')
        df1 = df1[period_filt] 
    elif periodo == 'Últimos 7 días':
        period_filt = [col for col in df1 if col.find('last 7 days')>=0]
        period_filt.insert(0, 'Name')
        df1 = df1[period_filt]
    else:
        period_filt = [col for col in df1 if col.find('last 24 hours')>=0]
        period_filt.insert(0, 'Name')
        df1 = df1[period_filt]        
    
    df1.sort_values(by=df1.columns[1], ascending=False, inplace = True)
    colnames = list(df1.columns)
    df1.rename({colnames[1]: 'Variable'}, axis=1, inplace = True)    
    
    
    # Definimos las gráficas del resultado interactivo
    fig = go.Figure(go.Bar(
                            x=df1['Variable'],
                            y=df1["Name"],
                            text=df1['Variable'],
                            textposition='outside',                                # Posición de la etiqueta de datos. En este caso a la derecha.
                            texttemplate='%{text}',                                # Formato de texto.
                            marker_color='rgb(191, 143, 0)',                      # Color de las barras.
                            orientation='h'                                        # Orientación de las barras. En este caso horizontal.
                          )
                   )
                        
    fig.update_layout(
                        barmode='stack',                                           # Tipo de barras. En este caso apiliadas.
                        yaxis_title="Pais",
                        xaxis_title="Número de "+str(estadistica),
                        title= 'Top 10 de países con mayor número de '+str(estadistica)+', '+ str(periodo),
                        xaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje x.
                        yaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje y.
                        plot_bgcolor='white',                                      # Modificamos el color del fondo del gráfico.
                        showlegend=False,                                           # Ocultamos la leyenda.
                        yaxis_range=[9.5, -.5],                                     # Mostramos los 10 primeros datos.
                        xaxis_range=[min(df1['Variable'])*.95, max(df1['Variable'])*1.2]
                     )
    
    return fig


def fig2():    
    #Seleccion del dataframe para la segunda grafica
    df2 = df[['Name', 'Deaths - cumulative total per 1 million population']]
    df2.sort_values(by=df2.columns[1], inplace=True, ascending=False)
    
    fig2 = go.Figure(go.Bar(
                            x=df2['Deaths - cumulative total per 1 million population'],
                            y=df2["Name"],
                            text=df2['Deaths - cumulative total per 1 million population'],
                            textposition='outside',                                # Posición de la etiqueta de datos. En este caso a la derecha.
                            texttemplate='%{text}',                                # Formato de texto.
                            marker_color='rgb(0,32,96)',                      # Color de las barras.
                            orientation='h'                                        # Orientación de las barras. En este caso horizontal.
                          )
                   )
    
    fig2.update_layout(
                        barmode='stack',                                           # Tipo de barras. En este caso apiliadas.
                        yaxis_title="Pais",
                        xaxis_title="Muertes por millon hab.",
                        title= 'Top 10 de países con mayor número de Muertes por millón de habitantes',
                        xaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje x.
                        yaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje y.
                        plot_bgcolor='white',                                      # Modificamos el color del fondo del gráfico.
                        showlegend=False,                                           # Ocultamos la leyenda.
                        yaxis_range=[9.5, -.5],                                     # Mostramos los 10 primeros datos.
                        xaxis_range=[min(df2['Deaths - cumulative total per 1 million population'])*.95, max(df2['Deaths - cumulative total per 1 million population'])*1.2]
                     )
    
    return fig2

def fig3():
   #Seleccion del dataframe para la tercera grafica
    df3 = df[['WHO Region', 'Cases - cumulative total']].groupby('WHO Region').sum('Cases - cumulative total').reset_index()
    df3.sort_values(by=['Cases - cumulative total'], inplace=True, ascending=False)
    
    fig3 = go.Figure(go.Bar(
                            x=df3['Cases - cumulative total'],
                            y=df3["WHO Region"],
                            text=df3['Cases - cumulative total'],
                            textposition='outside',                                # Posición de la etiqueta de datos. En este caso a la derecha.
                            texttemplate='%{text}',                                # Formato de texto.
                            marker_color='rgb(132,151,176)',                      # Color de las barras.
                            orientation='h'                                        # Orientación de las barras. En este caso horizontal.
                          )
                   )

    fig3.update_layout(
                        barmode='stack',                                           # Tipo de barras. En este caso apiliadas.
                        yaxis_title="Region",
                        xaxis_title="Casos Totales Acumulados",
                        title= 'Casos de COVID-19 por Región',
                        xaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje x.
                        yaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje y.
                        plot_bgcolor='white',                                      # Modificamos el color del fondo del gráfico.
                        showlegend=False,                                          # Ocultamos la leyenda.
                        yaxis_range=[6.5 , -.5],                                     # Mostramos los 10 primeros datos.
                        xaxis_range= [min(df3['Cases - cumulative total'])*.95, max(df3['Cases - cumulative total'])*1.2]
                     )

    return fig3

In [6]:
# YOUR CODE HERE
tablero_control = pn.Column(titulo,pn.Row(estadistica, periodo), pn.Row(pn.Column(funcion_interactiva,fig2,fig3)))

In [7]:
# YOUR CODE HERE
tablero_control.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] RadioButtonGroup(button_type='success', name='Estadistica', options=['Casos', 'Muertes'], value='Casos')
        [1] RadioButtonGroup(button_type='success', name='Periodo', options=['Histórico', ...], value='Histórico')
    [2] Row
        [0] Column
            [0] ParamFunction(function)
            [1] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure, name='interactive00111')
            [2] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure, name='interactive00117')

In [8]:
# YOUR CODE HERE
tablero_control.show()

Launching server at http://localhost:39835


## Créditos

**Autor(es)**: Juan David Reyes Jaimes, Diego Alejandro Cely Gomez

**Última actualización:** 26/08/2021